In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import math
import scipy.io as sio
import cv2
import matplotlib.image as mpimg
from matplotlib import path
from scipy.spatial.transform import Rotation 
from mpl_toolkits.mplot3d import Axes3D

In [3]:
DATA_DIR =  "data/data.mat"

data = sio.loadmat(DATA_DIR)['data']

In [4]:
x1 = data['x1'][0][0]; x2 = data['x2'][0][0]
x3 = data['x3'][0][0]; K = data['K'][0][0]
C = data['C'][0][0]; R = data['R'][0][0]
img1 = data['img1'][0][0].astype(np.uint8)
img2 = data['img2'][0][0].astype(np.uint8)
img3 = data['img3'][0][0].astype(np.uint8)

**Estimate Fundamental Matrix**

In [17]:
from estimate_fundamental_matrix import estimateFundamentalMatrix_RANSAC

F, inliers1, inliers2 = estimateFundamentalMatrix_RANSAC(x1,x2, num_iter=4)

In [19]:
F_, mask = cv2.findFundamentalMat(x1,x2,cv2.RANSAC)

**Visualisation of epipoles and epipolar lines**

**Essential Matrix from Fundamental Matrix**

In [53]:
def EssentialMatrixFromFundamentalMatrix(K1,F,K2):
    '''
       :param K1 : instrinsics of the first camera
       :param K2 : intrinsic of the second camera
       :param F: fundamental matrix
       
       Key Step is to ensure that singularity of the Essential Matrix. Enforce the singular va
       
       Returns:
           E : Essential Matrix
        
    
    
    '''
    E = K2.T @ F @ K1
    U, _ ,VT = np.linalg.svd(E)
    S = np.diag(np.array([1, 1, 0]))
    
    E = U @ S  @ VT
    
    return E

In [54]:
E = EssentialMatrixFromFundamentalMatrix(K,F_,K)

In [55]:
E

array([[ 0.02369549,  0.41183026, -0.13704612],
       [-0.23027232,  0.00581649, -0.96253342],
       [ 0.13225622,  0.90156466,  0.03481976]])

In [56]:
E

array([[ 0.02369549,  0.41183026, -0.13704612],
       [-0.23027232,  0.00581649, -0.96253342],
       [ 0.13225622,  0.90156466,  0.03481976]])

In [58]:
np.linalg.norm(E)

1.414213562373095